In [7]:
%pip install pymysql

  Using cached PyMySQL-1.1.0-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.0-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pandas

  Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install sqlalchemy

  Using cached SQLAlchemy-2.0.29-cp312-cp312-win_amd64.whl.metadata (9.8 kB)
  Using cached greenlet-3.0.3-cp312-cp312-win_amd64.whl.metadata (3.9 kB)
Using cached SQLAlchemy-2.0.29-cp312-cp312-win_amd64.whl (2.1 MB)
Using cached greenlet-3.0.3-cp312-cp312-win_amd64.whl (293 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
from database import engineconn
import pandas as pd

In [22]:
import pickle

In [9]:

db = engineconn()
products = pd.read_sql_query("SELECT * FROM product", db.engine)

In [10]:
member_id = 1
reports = pd.read_sql_query(f"SELECT skin_type FROM report WHERE member_id = {member_id}", db.engine)
print(reports.values.tolist())

[['NORMAL']]


In [15]:
# product_id들의 리스트
products_id = products['id']
print(len(products_id))
for i in range(10):
    print(products_id[i])

2913
1000000001588
1000000002326
1000000002839
1000000002848
1000000003296
1000000004434
1000000004925
1000000005336
1000000005485
1000000005766


In [12]:
# users
users = [i for i in range(100)]


In [16]:
# dummy my product dataFrame: 전체 랜덤
import random

idx_list = [i for i in range(900)]
user_id_list = [i % 50 for i in range(900)]

product_id_list = []

for i in range(1, 11):
    product_id_list += [products_id[i] for i in range(90 * (i - 1), 90 * i)]

rating_list = [random.randint(1,5) for i in range(900)]

my_products_all = pd.DataFrame({"user_id": user_id_list, "item_id": product_id_list, "rating": rating_list})

print(my_products_all)



     user_id        item_id  rating
0          0  1000000001588       5
1          1  1000000002326       1
2          2  1000000002839       4
3          3  1000000002848       4
4          4  1000000003296       1
..       ...            ...     ...
895       45  1000000158322       2
896       46  1000000158345       1
897       47  1000000158369       5
898       48  1000000158475       2
899       49  1000000158615       1

[900 rows x 3 columns]


In [24]:
from matrix_factorization import KernelMF

In [18]:
# 전체를 위한 추천
X = my_products_all[["user_id", "item_id"]]
y = my_products_all["rating"]

with open('all_X.pkl', 'wb') as f:
    pickle.dump(X, f)

matrix_fact_all = KernelMF(n_epochs=20, n_factors=100, verbose=1, lr=0.001, reg=0.005)
matrix_fact_all.fit(X, y)


Epoch  1 / 20  -  train_rmse: 1.3913535474743957
Epoch  2 / 20  -  train_rmse: 1.3860226262511492
Epoch  3 / 20  -  train_rmse: 1.3807535783966198
Epoch  4 / 20  -  train_rmse: 1.375540372413017
Epoch  5 / 20  -  train_rmse: 1.3703833850175782
Epoch  6 / 20  -  train_rmse: 1.3652790706955524
Epoch  7 / 20  -  train_rmse: 1.3602262395001703
Epoch  8 / 20  -  train_rmse: 1.355223604572539
Epoch  9 / 20  -  train_rmse: 1.350270851901056
Epoch  10 / 20  -  train_rmse: 1.345361508083711
Epoch  11 / 20  -  train_rmse: 1.3404981013653452
Epoch  12 / 20  -  train_rmse: 1.3356766552716586
Epoch  13 / 20  -  train_rmse: 1.3308967722050573
Epoch  14 / 20  -  train_rmse: 1.3261580798743131
Epoch  15 / 20  -  train_rmse: 1.3214572625242205
Epoch  16 / 20  -  train_rmse: 1.316791928943528
Epoch  17 / 20  -  train_rmse: 1.3121623628180379
Epoch  18 / 20  -  train_rmse: 1.30756641196298
Epoch  19 / 20  -  train_rmse: 1.303003304316743
Epoch  20 / 20  -  train_rmse: 1.2984712440140314


KernelMF(gamma=0.01, lr=0.001, n_epochs=20, reg=0.005)

In [19]:
# matrix_fact_all 저장

with open('matrix_fact_all.pkl', 'wb') as f:
    pickle.dump(matrix_fact_all, f)

In [20]:
# matrix_fact_all 사용
# 유저가 이미 리뷰를 남긴 제품들은 다시 추천을 받지 않는다.
member_id = 60
items_known = X.query("user_id == @member_id")["item_id"]

with open('matrix_fact_all.pkl', 'rb') as f:
    loaded_matrix_fact_all = pickle.load(f)


# # # amount개의 제품들을 추천받는다
recommended = loaded_matrix_fact_all.recommend(user=member_id, amount=20, items_known=items_known)[["item_id", "rating_pred"]]
print(recommended)

c:\mansman\S10P22E106\ML\skin_analysis\venv\Lib\site-packages\matrix_factorization\recommender_base.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan

           item_id  rating_pred
129  1000000129588     3.151435
531  1000000151047     3.151182
20   1000000148178     3.151146
97   1000000105646     3.151025
542  1000000130889     3.150829
335  1000000144041     3.150387
262  1000000115487     3.150307
599  1000000145282     3.150224
648  1000000147087     3.149950
250  1000000147357     3.149920
615  1000000157285     3.149886
820  1000000115500     3.149878
874  1000000104777     3.149512
863  1000000155995     3.149419
204  1000000129586     3.149409
697  1000000012755     3.149396
405  1000000152010     3.149259
750  1000000147641     3.149257
743  1000000113011     3.149166
235  1000000152280     3.149126


In [21]:
# dummy my product: 지성인 유저들
# skin_type_oily가 40 이상인 제품들의 리스트를 가져온다
oily_products_id = pd.read_sql_query("SELECT * FROM product WHERE skin_type_oily >= 40", db.engine)['id']
nonoily_products_id = pd.read_sql_query("SELECT * FROM product WHERE skin_type_oily <= 1", db.engine)['id']
# print(len(oily_products_id))
# print(len(nonoily_products_id))


total_list = []
# 지성인 유저들이 oily_products에 높은 평점을 준다 

first_half_users = users[:len(users)//2]
second_half_users = users[len(users) // 2:]

first_half_oily_products_id = oily_products_id[:len(oily_products_id) // 2]
second_half_oily_products_id = oily_products_id[len(oily_products_id) // 2:]

for u in first_half_users:
    for p in first_half_oily_products_id:
        total_list.append([u, p, random.randint(4, 5)])

for u in second_half_users:
    for p in second_half_oily_products_id:
        total_list.append([u, p, random.randint(4,5)])


# 지성인 유저들이 non_oily_products에 낮은 평점을 준다.
first_half_nonoily_products_id = nonoily_products_id[:len(nonoily_products_id) // 2]
second_half_nonoily_products_id = nonoily_products_id[len(nonoily_products_id) // 2:]

for u in first_half_users:
    for p in first_half_nonoily_products_id:
        total_list.append([u, p, random.randint(1,2)])

for u in second_half_users:
    for p in second_half_nonoily_products_id:
        total_list.append([u, p, random.randint(1,2)])


user_id_list = [l[0] for l in total_list]
product_id_list = [l[1] for l in total_list]
rating_list = [l[2] for l in total_list]


my_products_oily = pd.DataFrame({"user_id": user_id_list, "item_id": product_id_list, "rating": rating_list})

In [36]:
# 지성을 위한 추천
X = my_products_oily[["user_id", "item_id"]]
y = my_products_oily["rating"]

with open('oily_X.pkl', 'wb') as f:
    pickle.dump(X, f)

matrix_fact_oily = KernelMF(n_epochs=20, n_factors=100, verbose=1, lr=0.001, reg=0.005)
matrix_fact_oily.fit(X, y)

Epoch  1 / 20  -  train_rmse: 1.2995427267893165
Epoch  2 / 20  -  train_rmse: 1.2172743499621612
Epoch  3 / 20  -  train_rmse: 1.14185632541799
Epoch  4 / 20  -  train_rmse: 1.0727563150877844
Epoch  5 / 20  -  train_rmse: 1.0095118004649533
Epoch  6 / 20  -  train_rmse: 0.9517194774233638
Epoch  7 / 20  -  train_rmse: 0.8990067811469817
Epoch  8 / 20  -  train_rmse: 0.8510303561104327
Epoch  9 / 20  -  train_rmse: 0.8074827206442695
Epoch  10 / 20  -  train_rmse: 0.7680843075784798
Epoch  11 / 20  -  train_rmse: 0.7325483398247186
Epoch  12 / 20  -  train_rmse: 0.7006203592963053
Epoch  13 / 20  -  train_rmse: 0.6720438838680945
Epoch  14 / 20  -  train_rmse: 0.646570693796982
Epoch  15 / 20  -  train_rmse: 0.6239376806627297
Epoch  16 / 20  -  train_rmse: 0.6039220523853668
Epoch  17 / 20  -  train_rmse: 0.5862960766929837
Epoch  18 / 20  -  train_rmse: 0.5708318993031768
Epoch  19 / 20  -  train_rmse: 0.5572950759470169
Epoch  20 / 20  -  train_rmse: 0.5454950587052743


KernelMF(gamma=0.01, lr=0.001, n_epochs=20, reg=0.005)

In [23]:
# matrix_fact_oily 저장

with open('matrix_fact_oily.pkl', 'wb') as f:
    pickle.dump(matrix_fact_oily, f)

In [24]:
# matrix_fact_oily 사용
# 유저가 이미 리뷰를 남긴 제품들은 다시 추천을 받지 않는다.
member_id = 1
items_known = X.query("user_id == @member_id")["item_id"]

with open('matrix_fact_oily.pkl', 'rb') as f:
    loaded_matrix_fact_oily = pickle.load(f)


# # # amount개의 제품들을 추천받는다
recommended = loaded_matrix_fact_oily.recommend(user=member_id, amount=20, items_known=items_known)[["item_id", "rating_pred"]]
print(recommended)

           item_id  rating_pred
135  1000000199982     4.148424
140  1000000173116     4.148104
29   1000000198107     4.139055
30   1000000199154     4.099157
64   1000000188971     4.075816
116  1000000186191     4.074519
127  1000000181475     4.068273
105  1000000191768     4.035209
10   1000000186732     4.021530
35   1000000166448     4.016023
34   1000000189076     4.013637
83   1000000199777     3.997050
61   1000000182277     3.981162
81   1000000197319     3.960684
85   1000000167843     3.931656
62   1000000199723     3.929270
47   1000000185235     3.922929
114  1000000201981     3.921668
54   1000000198424     3.868394
32   1000000187924     3.866489


In [25]:
# dummy my product: 건성인 유저들
# skin_type_dry가 40 이상인 제품들의 리스트를 가져온다
dry_products_id = pd.read_sql_query("SELECT * FROM product WHERE skin_type_dry >= 50", db.engine)['id']
nondry_products_id = pd.read_sql_query("SELECT * FROM product WHERE skin_type_dry <= 20", db.engine)['id']
print(len(dry_products_id))
print(len(nondry_products_id))


total_list = []
# 지성인 유저들이 dry_products에 높은 평점을 준다 

first_half_users = users[:len(users)//2]
second_half_users = users[len(users) // 2:]

first_half_dry_products_id = dry_products_id[:len(dry_products_id) // 2]
second_half_dry_products_id = dry_products_id[len(dry_products_id) // 2:]

for u in first_half_users:
    for p in first_half_dry_products_id:
        total_list.append([u, p, random.randint(4, 5)])

for u in second_half_users:
    for p in second_half_dry_products_id:
        total_list.append([u, p, random.randint(4,5)])


# 지성인 유저들이 non_dry_products에 낮은 평점을 준다.
first_half_nondry_products_id = nondry_products_id[:len(nondry_products_id) // 2]
second_half_nondry_products_id = nondry_products_id[len(nondry_products_id) // 2:]

for u in first_half_users:
    for p in first_half_nondry_products_id:
        total_list.append([u, p, random.randint(1,2)])

for u in second_half_users:
    for p in second_half_nondry_products_id:
        total_list.append([u, p, random.randint(1,2)])


user_id_list = [l[0] for l in total_list]
product_id_list = [l[1] for l in total_list]
rating_list = [l[2] for l in total_list]


my_products_dry = pd.DataFrame({"user_id": user_id_list, "item_id": product_id_list, "rating": rating_list})

446
446


In [35]:
# 건성을 위한 추천
X = my_products_dry[["user_id", "item_id"]]
y = my_products_dry["rating"]

with open('dry_X.pkl', 'wb') as f:
    pickle.dump(X, f)

matrix_fact_dry = KernelMF(n_epochs=20, n_factors=100, verbose=1, lr=0.001, reg=0.005)
matrix_fact_dry.fit(X, y)

Epoch  1 / 20  -  train_rmse: 1.478546890210131
Epoch  2 / 20  -  train_rmse: 1.381251830960518
Epoch  3 / 20  -  train_rmse: 1.291162760115835
Epoch  4 / 20  -  train_rmse: 1.2074050855502452
Epoch  5 / 20  -  train_rmse: 1.1292133087141951
Epoch  6 / 20  -  train_rmse: 1.0559561245409916
Epoch  7 / 20  -  train_rmse: 0.9871503471305111
Epoch  8 / 20  -  train_rmse: 0.9224744884902215
Epoch  9 / 20  -  train_rmse: 0.861808369830879
Epoch  10 / 20  -  train_rmse: 0.8052060251105343
Epoch  11 / 20  -  train_rmse: 0.7529390575231099
Epoch  12 / 20  -  train_rmse: 0.7053575486683764
Epoch  13 / 20  -  train_rmse: 0.6628749168764531
Epoch  14 / 20  -  train_rmse: 0.6257952389711978
Epoch  15 / 20  -  train_rmse: 0.5942722670197935
Epoch  16 / 20  -  train_rmse: 0.5682023964723915
Epoch  17 / 20  -  train_rmse: 0.5471903807051274
Epoch  18 / 20  -  train_rmse: 0.5306824297866523
Epoch  19 / 20  -  train_rmse: 0.517984037242994
Epoch  20 / 20  -  train_rmse: 0.508367551855978


KernelMF(gamma=0.01, lr=0.001, n_epochs=20, reg=0.005)

In [27]:
# matrix_fact_dry 저장

with open('matrix_fact_dry.pkl', 'wb') as f:
    pickle.dump(matrix_fact_dry, f)

In [28]:
# matrix_fact_dry 사용
# 유저가 이미 리뷰를 남긴 제품들은 다시 추천을 받지 않는다.
member_id = 10
items_known = X.query("user_id == @member_id")["item_id"]

with open('matrix_fact_dry.pkl', 'rb') as f:
    loaded_matrix_fact_dry = pickle.load(f)


# # # amount개의 제품들을 추천받는다
recommended = loaded_matrix_fact_dry.recommend(user=member_id, amount=20, items_known=items_known)[["item_id", "rating_pred"]]
print(recommended)

           item_id  rating_pred
278  1000000191387     4.393742
22   1000000202518     4.342479
326  1000000189587     4.321011
386  1000000202471     4.314079
177  1000000192010     4.303988
377  1000000185367     4.297535
65   1000000191821     4.295958
44   1000000203023     4.292461
362  1000000188688     4.270460
183  1000000199483     4.256300
75   1000000198874     4.254931
413  1000000182425     4.253585
249  1000000200609     4.250820
436  1000000188789     4.249415
384  1000000202299     4.245535
404  1000000191259     4.244552
359  1000000189391     4.243417
104  1000000182647     4.243324
7    1000000189098     4.243012
432  1000000190390     4.240985


In [20]:
# dummy my product: 정상인 유저들을 위한 추천
ordinary_products_id = pd.read_sql_query("SELECT * FROM product WHERE skin_type_combination >= 65", db.engine)['id']
nonordinary_products_id = pd.read_sql_query("SELECT * FROM product WHERE skin_type_combination <= 40", db.engine)['id']
print(len(ordinary_products_id))
print(len(nonordinary_products_id))


total_list = []


first_half_users = users[:len(users)//2]
second_half_users = users[len(users) // 2:]

first_half_ordinary_products_id = ordinary_products_id[:len(ordinary_products_id) // 2]
second_half_ordinary_products_id = ordinary_products_id[len(ordinary_products_id) // 2:]

for u in first_half_users:
    for p in first_half_ordinary_products_id:
        total_list.append([u, p, random.randint(4, 5)])

for u in second_half_users:
    for p in second_half_ordinary_products_id:
        total_list.append([u, p, random.randint(4,5)])


# 
first_half_nonordinary_products_id = nonordinary_products_id[:len(nonordinary_products_id) // 2]
second_half_nonordinary_products_id = nonordinary_products_id[len(nonordinary_products_id) // 2:]

for u in first_half_users:
    for p in first_half_nonordinary_products_id:
        total_list.append([u, p, random.randint(1,2)])

for u in second_half_users:
    for p in second_half_nonordinary_products_id:
        total_list.append([u, p, random.randint(1,2)])


user_id_list = [l[0] for l in total_list]
product_id_list = [l[1] for l in total_list]
rating_list = [l[2] for l in total_list]


my_products_ordinary = pd.DataFrame({"user_id": user_id_list, "item_id": product_id_list, "rating": rating_list})


303
335


In [25]:
# 정상을 위한 추천
X = my_products_ordinary[["user_id", "item_id"]]
y = my_products_ordinary["rating"]

with open('ordinary_X.pkl', 'wb') as f:
    pickle.dump(X, f)

matrix_fact_ordinary = KernelMF(n_epochs=20, n_factors=100, verbose=1, lr=0.001, reg=0.005)
matrix_fact_ordinary.fit(X, y)

Epoch  1 / 20  -  train_rmse: 1.5071554600971409
Epoch  2 / 20  -  train_rmse: 1.4380546537361982
Epoch  3 / 20  -  train_rmse: 1.3720893162825865
Epoch  4 / 20  -  train_rmse: 1.3088344032238923
Epoch  5 / 20  -  train_rmse: 1.2479293587649423
Epoch  6 / 20  -  train_rmse: 1.1890578113844803
Epoch  7 / 20  -  train_rmse: 1.131945610674831
Epoch  8 / 20  -  train_rmse: 1.0763926881813686
Epoch  9 / 20  -  train_rmse: 1.0222643475360955
Epoch  10 / 20  -  train_rmse: 0.9695312650725898
Epoch  11 / 20  -  train_rmse: 0.9182612565361147
Epoch  12 / 20  -  train_rmse: 0.8686319971413713
Epoch  13 / 20  -  train_rmse: 0.8209307849185565
Epoch  14 / 20  -  train_rmse: 0.7755355598510658
Epoch  15 / 20  -  train_rmse: 0.732867891673435
Epoch  16 / 20  -  train_rmse: 0.6933754741735324
Epoch  17 / 20  -  train_rmse: 0.6574617106387192
Epoch  18 / 20  -  train_rmse: 0.625408249384124
Epoch  19 / 20  -  train_rmse: 0.5973669442995463
Epoch  20 / 20  -  train_rmse: 0.5733291882828433


KernelMF(gamma=0.01, lr=0.001, n_epochs=20, reg=0.005)

In [26]:

with open('matrix_fact_ordinary.pkl', 'wb') as f:
    pickle.dump(matrix_fact_ordinary, f)

In [42]:
import datetime

member_id = 1
age_column = pd.read_sql_query(f"SELECT birthday FROM member WHERE id = {member_id}", db.engine)

birthdate = age_column.values.tolist()[0][0]

# 현재 날짜 가져오기 (시뮬레이션)
current_date = datetime.date.today()

# 나이 계산
age = current_date.year - birthdate.year - ((current_date.month, current_date.day) < (birthdate.month, birthdate.day))

print("나이:", age)


AttributeError: 'list' object has no attribute 'year'